In [87]:
import numpy as np
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [47]:
class DecisionNode:
    def __init__(self, col, split, lchild, rchild):
        self.col = col
        self.split = split
        self.lchild = lchild
        self.rchild = rchild

    def predict(self, x_test):
        if x_test[self.col] < self.split: # Make decision based upon x_test[col] and split
            return self.lchild.predict(x_test)
        return self.rchild.predict(x_test) 

class LeafNode:
    def __init__(self, y, prediction):
        "Create leaf node from y values and prediction; prediction is mean(y) or mode(y)"
        self.n = len(y)
        self.prediction = prediction

    def predict(self, x_test):
        # return prediction
        return self.prediction

def gini(x):
    """
    Return the gini impurity score for values in y
    Assume y = {0,1}
    Gini = 1 - sum_i p_i^2 where p_i is the proportion of class i in y
    """
    if len(x) == 0:
        return 0
    unique_values, counts = np.unique(x, return_counts=True)
    p = np.sum((counts/len(x))**2) # proportion of class i in y
    return 1 - p 

    
def find_best_split(X, y, loss, min_samples_leaf):
    best = (-1, -1, loss(y))
    _, n_cols = X.shape
    # loop over all features 
    for col in range(0,n_cols): 
        feature_vals = X[:,col] # values in current feature
        candidates = np.random.choice(feature_vals, size=11) # pick 11 split candidates for current feature
        for i in candidates:
            # split the data into two groups, left and right, based on the split value, i, in column, col, of X, 
            y_left = y[feature_vals<i]
            y_right = y[feature_vals>=i]

            if len(y_left) < min_samples_leaf or len(y_right) < min_samples_leaf: 
                continue 
            l = ((len(y_left)*(loss(y_left))) + (len(y_right)*(loss(y_right)))) / (len(y_left) + len(y_right))
            if l == 0: 
                return col, i
            if l < best[2]:
                best = (col, i, l) # return column number, split value, loss 

    return best[0], best[1]
             
class DecisionTree621:
    def __init__(self, min_samples_leaf=1, loss=None):
        self.min_samples_leaf = min_samples_leaf
        self.loss = loss # loss function; either np.var for regression or gini for classification
        
    def fit(self, X, y):
        """
        Create a decision tree fit to (X,y) and save as self.root, the root of
        our decision tree, for either a classifier or regression.  Leaf nodes for classifiers
        predict the most common class (the mode) and regressions predict the average y
        for observations in that leaf.

        This function is a wrapper around fit_() that just stores the tree in self.root.
        """
        self.root = self.fit_(X, y)


    def fit_(self, X, y):
        """
        Recursively create and return a decision tree fit to (X,y) for
        either a classification or regression.  This function should call self.create_leaf(X,y)
        to create the appropriate leaf node, which will invoke either
        RegressionTree621.create_leaf() or ClassifierTree621.create_leaf() depending
        on the type of self.

        This function is not part of the class "interface" and is for internal use, but it
        embodies the decision tree fitting algorithm.

        (Make sure to call fit_() not fit() recursively.)
        """
        if len(X) < self.min_samples_leaf or len(np.unique(X))==1:
            return self.create_leaf(y)
        else:
            col, split = find_best_split(X,y,self.loss, self.min_samples_leaf) 
            if col == -1: 
                return self.create_leaf(y)
            
            left_child = self.fit_(X[X[:,col]<split], y[X[:,col]<split])
            right_child = self.fit_(X[X[:,col]>=split], y[X[:,col]>=split])

        return DecisionNode(col,split,left_child,right_child)

    def predict(self, X_test):
        """
        Make a prediction for each record in X_test and return as array.
        This method is inherited by RegressionTree621 and ClassifierTree621 and
        works for both without modification!
        """
        preds = [self.root.predict(x) for x in X_test]   
    
        return preds    

class RegressionTree621(DecisionTree621):
    def __init__(self, min_samples_leaf=1):
        super().__init__(min_samples_leaf, loss=np.var)

    def score(self, X_test, y_test):
        "Return the R^2 of y_test vs predictions for each record in X_test"
        return r2_score(y_test, self.predict(X_test))

    def create_leaf(self, y):
        """
        Return a new LeafNode for regression, passing y and mean(y) to
        the LeafNode constructor.
        """
        return LeafNode(y, np.mean(y))


class ClassifierTree621(DecisionTree621):
    def __init__(self, min_samples_leaf=1):
        super().__init__(min_samples_leaf, loss=gini)

    def score(self, X_test, y_test):
        "Return the accuracy_score() of y_test vs predictions for each record in X_test"
        return accuracy_score(y_test, self.predict(X_test))

    def create_leaf(self, y):
        """
        Return a new LeafNode for classification, passing y and mode(y) to
        the LeafNode constructor. Feel free to use scipy.stats to use the mode function.
        """
        return LeafNode(y, stats.mode(y, keepdims=False)[0])


---
## Apply on a sample data

### Classification dataset

In [56]:
from sklearn.datasets import load_iris

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model with the Iris dataset, train it, and evaluate its performance
model_iris = ClassifierTree621(min_samples_leaf=5)
model_iris.fit(X_train, y_train)
accuracy_iris = model_iris.score(X_test, y_test)
accuracy_iris


1.0

### Regression dataset

In [86]:
from sklearn.datasets import fetch_california_housing

# Load the California Housing Dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model with the California Housing dataset, train it, and evaluate its performance
model = RegressionTree621(min_samples_leaf=20)
model.fit(X_train, y_train)
accuracy = model.score(X_test, y_test)
accuracy


0.7110266874471061